In [7]:
from ultralytics import YOLO
import os
from pathlib import Path
import time
import json
import numpy as np
import cv2

# 載入訓練好的模型
model_path = r"C:\Users\hanaj\Desktop\HanaJ\Lab\YOLOgroundtruth\Floating_Plants_water.v6i.coco-segmentation\runs\water_segmentation\weights\best.pt"
model = YOLO(model_path)

# 設定測試圖片資料夾和結果輸出資料夾
test_folder = r"C:\Users\hanaj\Desktop\HanaJ\Lab\PlantPhoto"
output_folder = r"C:\Users\hanaj\Desktop\HanaJ\Lab\cropped_results"
mask_output_folder = r"C:\Users\hanaj\Desktop\HanaJ\Lab\cropped_masks"
json_output_folder = r"C:\Users\hanaj\Desktop\HanaJ\Lab\cropped_segmentation"

# 確保輸出資料夾存在
os.makedirs(output_folder, exist_ok=True)
os.makedirs(mask_output_folder, exist_ok=True)
os.makedirs(json_output_folder, exist_ok=True)

# 取得所有圖片檔案
image_files = list(Path(test_folder).glob("*.jpg")) + list(Path(test_folder).glob("*.JPG")) + list(Path(test_folder).glob("*.png"))
print(f"找到 {len(image_files)} 張圖片")

# 記錄開始時間
start_time = time.time()

# 處理每張圖片
for i, img_path in enumerate(image_files):
    print(f"處理圖片 {i+1}/{len(image_files)}: {img_path.name}")
    
    # 讀取原始圖片以獲取尺寸
    original_img = cv2.imread(str(img_path))
    img_height, img_width = original_img.shape[:2]
    
    # 進行預測
    results = model(img_path, conf=0.25, iou=0.45)
    
    # 儲存結果圖片
    output_path = Path(output_folder) / f"result_{img_path.name}"
    results[0].save(str(output_path))
    
    # 建立 instance segmentation 結果的 JSON 資料
    segmentation_data = {
        "filename": img_path.name,
        "width": img_width,
        "height": img_height,
        "instances": []
    }
    
    # 檢查是否有檢測到物件
    if results[0].masks is not None:
        masks = results[0].masks
        boxes = results[0].boxes
        
        # 處理每個檢測到的物件
        for j, (mask, box) in enumerate(zip(masks, boxes)):
            # 獲取分類和信心度
            cls = int(box.cls.item())
            conf = float(box.conf.item())
            class_name = results[0].names[cls]
            
            # 獲取遮罩資料
            mask_array = mask.data.cpu().numpy()[0]  # 取得 numpy 陣列
            
            # 儲存遮罩圖片
            mask_output_path = Path(mask_output_folder) / f"{img_path.stem}_mask_{j}.png"
            cv2.imwrite(str(mask_output_path), (mask_array * 255).astype(np.uint8))
            
            # 獲取輪廓
            contours = []
            binary_mask = (mask_array > 0.5).astype(np.uint8)
            opencv_contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # 轉換輪廓為多邊形格式
            for contour in opencv_contours:
                # 簡化輪廓點
                epsilon = 0.005 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)
                
                # 轉換為 [x1, y1, x2, y2, ...] 格式
                flattened = approx.flatten().tolist()
                if len(flattened) >= 6:  # 至少需要 3 個點 (6 個座標)
                    contours.append(flattened)
            
            # 計算面積
            area = float(cv2.contourArea(opencv_contours[0]) if opencv_contours else 0)
            
            # 獲取邊界框
            x1, y1, x2, y2 = box.xyxy.cpu().numpy()[0]
            bbox = [float(x1), float(y1), float(x2-x1), float(y2-y1)]  # [x, y, width, height]
            
            # 添加到 instance 資料
            instance = {
                "id": j,
                "category_id": cls,
                "category_name": class_name,
                "confidence": conf,
                "bbox": bbox,
                "area": area,
                "segmentation": contours
            }
            segmentation_data["instances"].append(instance)
        
        print(f"  ✓ 檢測到 {len(masks)} 個物件")
    else:
        print(f"  ✗ 未檢測到物件")
    
    # 儲存 JSON 檔案
    json_output_path = Path(json_output_folder) / f"{img_path.stem}_segmentation.json"
    with open(json_output_path, 'w', encoding='utf-8') as f:
        json.dump(segmentation_data, f, indent=2, ensure_ascii=False)

# 計算總處理時間
elapsed_time = time.time() - start_time
print(f"\n完成處理 {len(image_files)} 張圖片，耗時 {elapsed_time:.2f} 秒")
print(f"結果已儲存至:")
print(f"- 標註圖片: {output_folder}")
print(f"- 遮罩圖片: {mask_output_folder}")
print(f"- 分割資料: {json_output_folder}")


找到 838 張圖片
處理圖片 1/838: A1_20250402.JPG

image 1/1 C:\Users\hanaj\Desktop\HanaJ\Lab\PlantPhoto\A1_20250402.JPG: 480x640 2 water-plantss, 5.6ms
Speed: 2.3ms preprocess, 5.6ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)
  ✓ 檢測到 2 個物件
處理圖片 2/838: A1_20250406.jpg

image 1/1 C:\Users\hanaj\Desktop\HanaJ\Lab\PlantPhoto\A1_20250406.jpg: 480x640 2 water-plantss, 5.1ms
Speed: 1.3ms preprocess, 5.1ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)
  ✓ 檢測到 2 個物件
處理圖片 3/838: A1_20250409.JPG

image 1/1 C:\Users\hanaj\Desktop\HanaJ\Lab\PlantPhoto\A1_20250409.JPG: 480x640 2 water-plantss, 5.8ms
Speed: 1.9ms preprocess, 5.8ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)
  ✓ 檢測到 2 個物件
處理圖片 4/838: A1_20250410.JPG

image 1/1 C:\Users\hanaj\Desktop\HanaJ\Lab\PlantPhoto\A1_20250410.JPG: 480x640 5 water-plantss, 5.0ms
Speed: 2.2ms preprocess, 5.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
  ✓ 檢測到 5 個物件
處理圖片 5/838: A1_20250411.J